In [1]:
import os

os.chdir('..')
print(os.getcwd())

/workspace/LLaSA


In [2]:
import importlib

#importlib.reload(importlib.import_module("model.llasa_arch"))
importlib.reload(importlib.import_module("model.llasa_processor"))
#importlib.reload(importlib.import_module("utils.dataloader"))

/opt/conda/envs/llasa/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'model.llasa_processor' from '/workspace/LLaSA/model/llasa_processor.py'>

In [3]:
# reload module 

import torch
from transformers import BitsAndBytesConfig, LlavaNextVideoForConditionalGeneration
from model.llasa_arch import LLaSA
from model.llasa_processor import LlavaNextVideoProcessor


quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

#del model

processor = LlavaNextVideoProcessor.from_pretrained("/workspace/LLaSA/checkpoints/LLaVA-NeXT-Video-7B-hf", cache_dir='/workspace/LLaSA/checkpoints')
model = LLaSA.from_pretrained(
    "/workspace/LLaSA/checkpoints/LLaVA-NeXT-Video-7B-hf",
    quantization_config=quantization_config,
    device_map='auto',
    cache_dir = '/workspace/LLaSA/checkpoints'
)


Unrecognized keys in `rope_scaling` for 'rope_type'='linear': {'type'}
Loading checkpoint shards: 100%|██████████| 3/3 [00:09<00:00,  3.00s/it]
Some weights of LLaSA were not initialized from the model checkpoint at /workspace/LLaSA/checkpoints/LLaVA-NeXT-Video-7B-hf and are newly initialized: ['seg_projector.linear.bias', 'seg_projector.linear.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [5]:
from utils.dataloader import Dataset_A2D
ds = Dataset_A2D(processor = processor, start_idx=0, end_idx=2000)

In [11]:
ds =ds.with_format("torch")

AttributeError: 'Dataset_A2D' object has no attribute 'with_format'

In [7]:
from peft import LoraConfig


lora_config = LoraConfig(
    target_modules=["q_proj", "k_proj"],
    init_lora_weights=False
)

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

model.add_adapter(lora_config, adapter_name="Llasa_adapter")
model.set_adapter('Llasa_adapter')

In [8]:
from transformers import TrainingArguments, logging, Trainer
from model.llasa_processor import LlavaNextVideoDataCollatorWithPadding

default_args = {
    "output_dir": "tmp",
    "eval_strategy": "steps",
    "num_train_epochs": 1,
    "log_level": "error",
    "report_to": "none",
    "dataloader_pin_memory":False,
}

logging.set_verbosity_error()


training_args = TrainingArguments(per_device_train_batch_size=1, **default_args)
trainer = Trainer(
    model=model, 
    args=training_args, 
    data_collator = LlavaNextVideoDataCollatorWithPadding(processor=processor)
    train_dataset=ds
)
result = trainer.train()
print_summary(result)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Expanding inputs for image.video tokens in LLaVa-NeXT-Video should be done in processing. Please add `patch_size` and `vision_feature_select_strategy` to the model's processing config or set directly with `processor.patch_size = {{patch_size}}` and processor.vision_feature_select_strategy = {{vision_feature_select_strategy}}`. Using processors without these attributes in the config is deprecated and will throw an error in v4.47.


TypeError: 'NoneType' object is not subscriptable